# Custom Transformer

In [1]:
from sklearn import set_config; set_config(display='diagram')

👇 Consider the following dataset

In [2]:
import pandas as pd

data = pd.read_csv("data.csv")

data.head()

,customer_state,seller_state,product_weight_g,product_length_cm,product_height_cm,product_width_cm,days_until_delivery
0,RJ,SP,1825,53,10,40,9
1,RJ,SP,700,65,18,28,9
2,RJ,SP,1825,53,10,40,11
3,RJ,SP,1825,53,10,40,12
4,RJ,SP,1825,53,10,40,14


Each observation of the dataset represents an item being delivered from a  `seller_state` to a `customer_state`. The columns describe the size and weight of each item. The target is the number of days between the order and the delivery.

👇 In a pipeline:

- Engineer a 'volume' feature from the dimensions features
- Preserve the original product dimensions features for training
- Scale all numerical features
- Encode the categorical features
- Train a default `Ridge` regression and cross_validate its score on the train set. Low r2 score are expected.

Use your pipeline to predict the delivery of the following order

In [3]:
X = data.drop(columns='days_until_delivery')
y = data['days_until_delivery']

In [ ]:
X.dtypes

In [4]:
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer

from sklearn.linear_model import Ridge

#Create X_Train , y_train, X_test, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

In [5]:
from sklearn.preprocessing import FunctionTransformer

volume_constructor = FunctionTransformer(lambda df: 
                                                pd.DataFrame(df('product_length_cm') * df('product_height_cm') * df('product_width_cm')))





In [8]:
from sklearn.preprocessing import MinMaxScaler

# Impute then Scale for numerical variables: 
num_transformer = Pipeline([
    ('scaler', MinMaxScaler())])

from sklearn.preprocessing import OneHotEncoder
# Encode categorical variables
cat_transformer = OneHotEncoder(handle_unknown='ignore')



In [11]:
#Select features automatically according to their type
from sklearn.compose import make_column_selector


preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, make_column_selector(dtype_include=['float64'])),
    ('cat_transformer', cat_transformer, make_column_selector(dtype_include=['object']))],
    remainder='passthrough')

In [12]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num_transformer',
                                 Pipeline(steps=[('scaler', MinMaxScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f1542b4e8e0>),
                                ('cat_transformer',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f1542b4e550>)])

In [ ]:
volume_transformer = Pipeline([
    ('imputer', KNNImputer()),
    ('scaler', MinMaxScaler())])

In [ ]:
preprocess_volume = Pipeline([
    ('vol_transformer', FunctionTransformer(vol)), 
    ('scaler_vol', MinMaxScaler())
    
])

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer

# Create a transformer that multiplies three columns
volume_constructor = FunctionTransformer(lambda df: 
                                                pd.DataFrame(df('product_length_cm') * df('product_height_cm') * df('product_width_cm')))


union = FeatureUnion([
    ('preprocess', preprocessor), 
    ('volume', volume_constructor),
    ])

#union.fit(X_train)
union


In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

# Impute then Scale for numerical variables: 
num_transformer = Pipeline([
    ('imputer', KNNImputer()),
    ('scaler', MinMaxScaler())])

from sklearn.preprocessing import OneHotEncoder
# Encode categorical variables
cat_transformer = OneHotEncoder(handle_unknown='ignore')


from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer

# Create a transformer that multiplies three columns
volume_constructor = FunctionTransformer(lambda df: pd.DataFrame(df('product_length_cm') * df('product_height_cm') 
                                                                 * df('product_width_cm')))
volume_transformer = Pipeline([
    ('imputer', KNNImputer()),
    ('scaler', MinMaxScaler())])


#Select features automatically according to their type
from sklearn.compose import make_column_selector


preprocessor = ColumnTransformer([
    ('num_transformer', num_transformer, make_column_selector(dtype_include=['float64'])),
    ('cat_transformer', cat_transformer, make_column_selector(dtype_include=['object']))],
    ('volume', volume_constructor,['product_length_cm','product_height_cm','product_width_cm'] ),
    remainder='passthrough')



final_pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('linear_regression', Ridge())])

final_pipe

In [ ]:
new_data = pd.read_csv("data_new.csv")
new_data

<details><summary>Hints</summary>

- There are many ways to create your preprocessed matrix (using `ColumnTransformer` and/or `FeatureUnion`). 
    
- If your transformed feature matrix look wierd, it may be stored as "sparse" by the default behavior of `OneHotEncoder(sparse=True)`. Use `.todense()` to turn it back to a dense matrix

</details>